In [1]:
import pandas as pd, numpy as np

In [6]:
## Bring in gene changes file
gc = pd.read_csv('/home/croth/Documents/DirtyDozen/DATA/gene_changes.csv.gz')
gc['Parent'] = gc['Gene']
gc['Gene'] = [g.split('-t26')[0] for g in gc.Parent]
gc.head()
BF = gc[(gc.Strain.isin(['BT22','FTC555']))].sort_values(['Gene','Strain'])

In [13]:
## Bring in GFF file
gffpath = '/home/croth/Documents/DirtyDozen/REF/FungiDB-46_CneoformansH99.gff'

descriptions = ['hypothetical protein','unspecified product']

names = ["Seqid", "Source", "Type", "Start", "End", "Score", 
         "Strand", "Phase", "Attributes"]
dtype = ["str","str","str","int","int","str","str","str","str"]

gff = pd.read_csv(gffpath,
                   comment="#",header=None,
                  delimiter="\t",names=names,
                    dtype=dict(zip(names,dtype)))

foi = ['gene','three_prime_UTR','five_prime_UTR','CDS']

gff = gff[(gff.Type.isin(foi))]

gff['Gene'] = ['CNAG'+a.split(';')[0].split('-')[0].split('CNAG')[-1] 
               for a in gff.Attributes]

gff['Parent'] = [a.split('Parent=')[-1].split(';')[0].split('ID=')[-1] 
                 for a in gff.Attributes ]

genes = gff[(gff.Type=='gene')].copy()
genes['Description'] = [a.split('description=')[-1].split('%2C')[0] 
                        for a in genes.Attributes]

genes['Chrom'] = [int(s.split('0038')[-1].split('.')[0])-19 for s in genes.Seqid.tolist()]

In [15]:
genes.head()

,Seqid,Source,Type,Start,End,Score,Strand,Phase,Attributes,Gene,Parent,Description,Chrom
0,CP003824.1,EuPathDB,gene,409610,410982,.,+,.,ID=CNAG_01397;description=mitochondrial import...,CNAG_01397,CNAG_01397,mitochondrial import inner membrane translocas...,5
12,CP003833.2,EuPathDB,gene,591505,596748,.,+,.,ID=CNAG_05539;description=hypothetical protein,CNAG_05539,CNAG_05539,hypothetical protein,14
60,CP003832.1,EuPathDB,gene,145290,146476,.,-,.,ID=CNAG_06307;description=hypothetical protein,CNAG_06307,CNAG_06307,hypothetical protein,13
73,CP003821.1,EuPathDB,gene,17403,19442,.,+,.,ID=CNAG_06796;description=serine/arginine repe...,CNAG_06796,CNAG_06796,serine/arginine repetitive matrix protein 1,2
85,CP003822.1,EuPathDB,gene,1200859,1202351,.,+,.,ID=CNAG_07556;description=hypothetical protein,CNAG_07556,CNAG_07556,hypothetical protein,3


In [16]:
chrom_8_qtl = (648243,857460)

In [55]:
BFg = BF.merge(genes[['Gene','Chrom','Start','End','Description']])

In [56]:
BF8 = BFg[(BFg.Chrom==8)]
BF8.shape

(632, 13)

In [57]:
BF8.to_csv('/home/croth/Bt22xFtc555-1/NOTES/CHROM8_GENES.csv',index=False)

In [23]:
QTL = BF8[(BF8.Start>=np.min(chrom_8_qtl)) & (BF8.End<=np.max(chrom_8_qtl))]

In [30]:
QTL.Gene.unique().shape

(45,)

In [24]:
QTL.shape

(102, 13)

In [48]:
sg_in_xl280 = 'CNAG_03342'
QTL[(QTL.Gene==sg_in_xl280)]

,Strain,Gene,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Parent,Chrom,Start,End,Description
3777,BT22,CNAG_03342,569.0,569,569,1,1,10,CNAG_03342-t26_1,8,692706,694766,tRNA (guanine-N(7)-)-methyltransferase subunit...
3778,FTC555,CNAG_03342,569.0,569,569,1,1,8,CNAG_03342-t26_1,8,692706,694766,tRNA (guanine-N(7)-)-methyltransferase subunit...


In [47]:
QTL.groupby('Parent').agg(np.diff).abs()['Nonsyn'].sort_values().tail()

Parent
CNAG_03341-t26_1    2.0
CNAG_03369-t26_1    2.0
CNAG_03340-t26_1    3.0
CNAG_03368-t26_2    5.0
CNAG_03368-t26_1    5.0
Name: Nonsyn, dtype: float64

In [45]:
mac1 = 'CNAG_07724' ## CNG02270
QTL[(QTL.Gene==mac1)]

,Strain,Gene,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Parent,Chrom,Start,End,Description
8130,BT22,CNAG_07724,1048.0,1048,1048,1,1,4,CNAG_07724-t26_1,8,721568,725406,Metal-binding regulatory protein CUF1
8131,FTC555,CNAG_07724,1048.0,1048,1048,1,1,5,CNAG_07724-t26_1,8,721568,725406,Metal-binding regulatory protein CUF1


In [44]:
QTL[(QTL.Parent=='CNAG_03368-t26_1')]

,Strain,Gene,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Parent,Chrom,Start,End,Description
3802,BT22,CNAG_03368,2208.0,2208,2208,1,1,15,CNAG_03368-t26_1,8,766081,774784,microtubule Associated protein
3804,FTC555,CNAG_03368,2208.0,2208,2208,1,1,20,CNAG_03368-t26_1,8,766081,774784,microtubule Associated protein


In [53]:
QTL[(QTL.Nonsyn>0)].sort_values('Nonsyn')[-20:-10]

,Strain,Gene,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Parent,Chrom,Start,End,Description
3772,FTC555,CNAG_03339,492.0,492,493,1,0,7,CNAG_03339-t26_2,8,684296,687151,biotin transporter
3795,BT22,CNAG_03366,738.0,738,738,1,1,7,CNAG_03366-t26_1,8,755992,763369,C2H2 type zinc finger transcription factor
3796,BT22,CNAG_03366,738.0,738,738,1,1,7,CNAG_03366-t26_2,8,755992,763369,C2H2 type zinc finger transcription factor
3797,FTC555,CNAG_03366,738.0,738,738,1,1,8,CNAG_03366-t26_1,8,755992,763369,C2H2 type zinc finger transcription factor
3798,FTC555,CNAG_03366,738.0,738,738,1,1,8,CNAG_03366-t26_2,8,755992,763369,C2H2 type zinc finger transcription factor
3778,FTC555,CNAG_03342,569.0,569,569,1,1,8,CNAG_03342-t26_1,8,692706,694766,tRNA (guanine-N(7)-)-methyltransferase subunit...
3805,BT22,CNAG_03369,1172.0,1172,1172,1,1,9,CNAG_03369-t26_1,8,776302,780325,WEE protein kinase
3777,BT22,CNAG_03342,569.0,569,569,1,1,10,CNAG_03342-t26_1,8,692706,694766,tRNA (guanine-N(7)-)-methyltransferase subunit...
3836,FTC555,CNAG_03392,1150.0,1150,1150,1,1,11,CNAG_03392-t26_1,8,832538,837616,DNA supercoiling protein
3835,BT22,CNAG_03392,1150.0,1150,1150,1,1,12,CNAG_03392-t26_1,8,832538,837616,DNA supercoiling protein


In [31]:
QTL[(QTL.Nonsyn>0)]

,Strain,Gene,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Parent,Chrom,Start,End,Description
3755,BT22,CNAG_03325,697.0,697,697,1,1,4,CNAG_03325-t26_1,8,651584,654822,ChAPs family protein
3756,FTC555,CNAG_03325,697.0,697,697,1,1,5,CNAG_03325-t26_1,8,651584,654822,ChAPs family protein
3757,BT22,CNAG_03326,942.0,942,942,1,1,5,CNAG_03326-t26_1,8,655318,659025,Chitin synthase
3758,FTC555,CNAG_03326,942.0,942,942,1,1,4,CNAG_03326-t26_1,8,655318,659025,Chitin synthase
3759,BT22,CNAG_03329,538.0,538,539,1,1,437,CNAG_03329-t26_1,8,662898,664806,PHD-finger protein
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8133,BT22,CNAG_07725,582.0,582,582,1,1,17,CNAG_07725-t26_2,8,781856,784029,HMG box factor
8134,FTC555,CNAG_07725,578.0,578,578,1,1,18,CNAG_07725-t26_1,8,781856,784029,HMG box factor
8135,FTC555,CNAG_07725,582.0,582,582,1,1,18,CNAG_07725-t26_2,8,781856,784029,HMG box factor
8136,BT22,CNAG_07728,113.0,113,113,1,1,2,CNAG_07728-t26_1,8,847527,848074,solute carrier family 39 (zinc transporter)


In [29]:
QTL[(QTL.Altstop!=1)]

,Strain,Gene,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Parent,Chrom,Start,End,Description
3769,BT22,CNAG_03339,495.0,495,496,1,0,6,CNAG_03339-t26_1,8,684296,687151,biotin transporter
3770,BT22,CNAG_03339,492.0,492,493,1,0,6,CNAG_03339-t26_2,8,684296,687151,biotin transporter
3771,FTC555,CNAG_03339,495.0,495,496,1,0,7,CNAG_03339-t26_1,8,684296,687151,biotin transporter
3772,FTC555,CNAG_03339,492.0,492,493,1,0,7,CNAG_03339-t26_2,8,684296,687151,biotin transporter


In [28]:
QTL[(QTL.Altstop>1)]

,Strain,Gene,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Parent,Chrom,Start,End,Description


In [26]:
QTL[(QTL.Refstop>1)]

,Strain,Gene,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Parent,Chrom,Start,End,Description
